# Marchantia Image Processing 

In [ ]:
# MODULE IMPORT
import os # For manipulating system information (file & folder paths and related)
from skimage import io # For importing and saving images
from skimage import exposure # For improving contrast and brightness in an image
from skimage import img_as_float # To transform an 8 bit image to floating points
from skimage.color import * # For color manipulation and transforming
import matplotlib.pyplot as plt # for plotting all the figures
from skimage import filters # for thresholding algorithms
from skimage import morphology # for dilation, erosion and other binary transforms
import numpy as np # for array manipulation
from IPython.display import clear_output # Used to avoid output cluttering
from tkinter import Tcl # Module used for intelligent sorting of lists
import scipy.ndimage as ndi 
%matplotlib inline
# Required for viewing images in silico

# The modules listed below may or may not be necessary depending on the code executed

#import re # Regex module for powerful manipulation of strings
# import skimage.feature as skfeat # for detecting ROIs and OOIs
# from math import *
# from scipy import stats # Module used for statistical analysis of data obtained from images

### Search the folder with the images

The procedure for extracting the filepath and the filename is as follows:

Inside the main folder (MP_5DAYS) are a number of subfolders; these subfolders contain either only brightfield images (in case of WT) or contain 2 subfolders, named campo claro & fluo, and the corresponding
brightfield or fluorescent photos are within these folders. This cell analyses the folders according to their name, and extracts the filepath and filename. These are sorted alphabetically, by day and by 
plant number. The resulting arrays contain all the filepaths and filenames, ordered by day, and within the day ordered alphabetically.

As an example, after running this cell one would obtain:

```python
all_files[day][filepaths]
all_names[day][filename]
```

As the filepath and filename arrays are sorted, the index for a specific photo in the filepath array corresponds to the same index in the filename array

In [ ]:
# First custom function; used for the extraction of every image file and their corresponding name

def filesAndNames(days):

    '''
    This function is used for extracting the full filepath for .TIF images from specified paths. Function takes as input:

    Search directory is the root directory where the search is going to be conducted; Inside this directory there should be 
    various folders corresponding to each of the plant variants (WT, BFP, Venus, etc.). Inside each of these folders there 
    should be DAYS subfolders, corresponding to each of the days the experiment was conducted.

    Days - Number of days the experiment was conducted. Must be of type Int.

    It produces as output:

    all_files - Array of arrays, each of which contains every image found within the folders
    all_names - Array of arrays, each of which contains the NAME of every image found within the folders
    '''
    
    global paths
    
    operating_system = str(input('Please enter the OS of your computer ("WINDOWS" or "MAC OS",requiered for correct searching of files): '))
    search_directory = str(input('Please enter the directory where the image folders and files are: '))
    os.chdir(search_directory)
    paths = [os.path.join(search_directory, f) for f in os.listdir(search_directory)]
    paths.sort()
    

    all_files = [[] for i in range(days)]
    all_names = [[] for i in range(days)]
    
    if operating_system.upper()=='WINDOWS':
        
        for i in range(len(paths)):

            if 'WT' in paths[i]:

                print('This folder contains wild type Mp photos')
                os.chdir(paths[i]) #cambiar directorio a donde esta el wt
                files = filter(os.path.isfile, os.listdir(paths[i])) #solo los que sean archivos
                files = [os.path.join(paths[i], f) for f in files] #guarda los path
                files = list(map(lambda x: x.replace("\\", "/"), files))
                names = []
                for file in files:

                    filename, ext = os.path.splitext(file) #conseguir extension del archivo

                    # CHANGE THE EXTENSION TO TIFF OR TIF FOR IMAGE FILES
                    if ext == '.tif':
                        names.append(filename.split(search_directory+'WT/')[1])
                    else:
                        continue

                          # We sort all files in the folder

                sorted_files = Tcl().call('lsort', '-dict', files)
                sorted_names = Tcl().call('lsort', '-dict', names)

                for i in range(0, len(sorted_files)):

                    filename, ext = os.path.splitext(sorted_files[i])
                    first_split = filename.split(' ', 1)
                    second_split = first_split[1].split(' ')[0] # This gives us the specific day

                    if second_split == '0':
                        all_files[0].append(sorted_files[i])
                        all_names[0].append(sorted_names[i])
                        continue

                    elif second_split == '1':
                        all_files[1].append(sorted_files[i])
                        all_names[1].append(sorted_names[i])
                        continue

                    elif second_split == '2':
                        all_files[2].append(sorted_files[i])
                        all_names[2].append(sorted_names[i])
                        continue

                    elif second_split == '3':
                        all_files[3].append(sorted_files[i])
                        all_names[3].append(sorted_names[i])
                        continue

                    elif second_split == '4':
                        all_files[4].append(sorted_files[i])
                        all_names[4].append(sorted_names[i])
                        continue
            else:

                print('This folder contains brightfield and fluorescence photos')
                os.chdir(paths[i])
                folders = filter(os.path.isdir, os.listdir(paths[i]))
                folders = [os.path.join(paths[i], f) for f in folders]
                names = []

                # Since for all transformed plants we have two subfolders within the main folder,
                # we must loop through all brightfield photos in the brightfield folder and all
                # fluorescent photos in the fluorescent folder    

                for folder in folders:

                    if 'Fluo' in folder:

                        print('Analyzing fluorescent picture folder')

                        os.chdir(folder)
                        files = filter(os.path.isfile, os.listdir(folder))
                        files = [os.path.join(folder, f) for f in files]
                        files = list(map(lambda x: x.replace("\\", "/"), files))
                        names = []

                        for file in files:

                            filename, ext = os.path.splitext(file)
                            #CHANGE THE EXTENSION TO TIFF OR TIF FOR IMAGE FILES
                            if ext == '.tif':
                                name = file.rsplit('/', 2)[2]
                                name = name.rsplit('.')[0]
                                names.append(name)
                            else:
                                continue
                                # We sort all files in the folder

                        sorted_files = Tcl().call('lsort', '-dict', files)
                        sorted_names = Tcl().call('lsort', '-dict', names)

                        for i in range(0, len(sorted_files)):

                            filename, ext = os.path.splitext(sorted_files[i])
                            first_split = filename.split(' ', 1)
                            second_split = first_split[1].split(' ')[1] # This gives us the specific day

                            if second_split == '0':
                                all_files[0].append(sorted_files[i])
                                all_names[0].append(sorted_names[i])
                                continue

                            elif second_split == '1':
                                all_files[1].append(sorted_files[i])
                                all_names[1].append(sorted_names[i])
                                continue

                            elif second_split == '2':
                                all_files[2].append(sorted_files[i])
                                all_names[2].append(sorted_names[i])
                                continue

                            elif second_split == '3':
                                all_files[3].append(sorted_files[i])
                                all_names[3].append(sorted_names[i])
                                continue

                            elif second_split == '4':
                                all_files[4].append(sorted_files[i])
                                all_names[4].append(sorted_names[i])
                                continue

                    elif 'Campo' in folder or 'Claro' in folder:

                        print('Analizing brightfield picture folder')

                        os.chdir(folder)
                        files = filter(os.path.isfile, os.listdir(folder))
                        files = [os.path.join(folder, f) for f in files]
                        files = list(map(lambda x: x.replace("\\", "/"), files))
                        names = []
                        for file in files:

                            filename, ext = os.path.splitext(file)
                            #CHANGE THE EXTENSION TO TIFF OR TIF FOR IMAGE FILES
                            if ext == '.tif':
                                name = file.rsplit('/', 2)[2]
                                name = name.rsplit('.')[0]
                                names.append(name)
                            else:
                                continue

                        # We sort all files in the folder

                        sorted_files = Tcl().call('lsort', '-dict', files)
                        sorted_names = Tcl().call('lsort', '-dict', names)



                        for i in range(0, len(sorted_files)):

                            filename, ext = os.path.splitext(sorted_files[i])
                            first_split = filename.split(' ', 1)
                            second_split = first_split[1].split(' ')[1] # This gives us the specific day

                        #    print('File:', filename, '\nDay:', second_split)

                            if second_split == '0':
                                all_files[0].append(sorted_files[i])
                                all_names[0].append(sorted_names[i])
                                continue

                            elif second_split == '1':
                                all_files[1].append(sorted_files[i])
                                all_names[1].append(sorted_names[i])
                                continue

                            elif second_split == '2':
                                all_files[2].append(sorted_files[i])
                                all_names[2].append(sorted_names[i])
                                continue

                            elif second_split == '3':
                                all_files[3].append(sorted_files[i])
                                all_names[3].append(sorted_names[i])
                                continue

                            elif second_split == '4':
                                all_files[4].append(sorted_files[i])
                                all_names[4].append(sorted_names[i])
                                continue

                clear_output(wait=True) # Comment out this line if you would like to see the all the files detected

        return all_files, all_names
        
    elif operating_system.upper()=='MAC OS' or operating_system.upper()=='MAC' or operating_system.upper()=='MACOS':

        for i in range(len(paths)):

            # If the folder contains WT type photos, we only have to extract the type, day and number of the plant

            if 'DS_Store' in paths[i]:

                continue

            elif 'WT' in paths[i]:

                print('This folder contains wild type Mp photos')
                os.chdir(paths[i])
                files = filter(os.path.isfile, os.listdir(paths[i]))
                files = [os.path.join(paths[i], f) for f in files]
                names = []
                for file in files:

                    filename, ext = os.path.splitext(file)
                    #CHANGE THE EXTENSION TO TIFF OR TIF FOR IMAGE FILES
                    if ext == '.tif':
                        name = file.rsplit('/', 2)[2]
                        name = name.rsplit('.')[0]
                        names.append(name)
                    else:
                        continue

                # We sort all files in the folder

                sorted_files = Tcl().call('lsort', '-dict', files)
                sorted_names = Tcl().call('lsort', '-dict', names)

                for i in range(0, len(sorted_files)):

                    filename, ext = os.path.splitext(sorted_files[i])
                    first_split = filename.split(' ', 1)
                    second_split = first_split[1].split(' ')[0] # This gives us the specific day

                    if second_split == '0':
                        all_files[0].append(sorted_files[i])
                        all_names[0].append(sorted_names[i])
                        continue

                    elif second_split == '1':
                        all_files[1].append(sorted_files[i])
                        all_names[1].append(sorted_names[i])
                        continue

                    elif second_split == '2':
                        all_files[2].append(sorted_files[i])
                        all_names[2].append(sorted_names[i])
                        continue

                    elif second_split == '3':
                        all_files[3].append(sorted_files[i])
                        all_names[3].append(sorted_names[i])
                        continue

                    elif second_split == '4':
                        all_files[4].append(sorted_files[i])
                        all_names[4].append(sorted_names[i])
                        continue

            else:

                print('This folder contains brightfield and fluorescence photos')
                os.chdir(paths[i])
                folders = filter(os.path.isdir, os.listdir(paths[i]))
                folders = [os.path.join(paths[i], f) for f in folders]
                names = []

                # Since for all transformed plants we have two subfolders within the main folder,
                # we must loop through all brightfield photos in the brightfield folder and all
                # fluorescent photos in the fluorescent folder

                for folder in folders:

                    if 'Fluo' in folder:

                        print('Analyzing fluorescent picture folder')

                        os.chdir(folder)
                        files = filter(os.path.isfile, os.listdir(folder))
                        files = [os.path.join(folder, f) for f in files]
                        names = []
                        for file in files:

                            filename, ext = os.path.splitext(file)
                            #CHANGE THE EXTENSION TO TIFF OR TIF FOR IMAGE FILES
                            if ext == '.tif':
                                name = file.rsplit('/', 2)[2]
                                name = name.rsplit('.')[0]
                                names.append(name)
                            else:
                                continue

                        # We sort all files in the folder

                        sorted_files = Tcl().call('lsort', '-dict', files)
                        sorted_names = Tcl().call('lsort', '-dict', names)

                        for i in range(0, len(sorted_files)):

                            filename, ext = os.path.splitext(sorted_files[i])
                            first_split = filename.split(' ', 1)
                            second_split = first_split[1].split(' ')[1] # This gives us the specific day

                            if second_split == '0':
                                all_files[0].append(sorted_files[i])
                                all_names[0].append(sorted_names[i])
                                continue

                            elif second_split == '1':
                                all_files[1].append(sorted_files[i])
                                all_names[1].append(sorted_names[i])
                                continue

                            elif second_split == '2':
                                all_files[2].append(sorted_files[i])
                                all_names[2].append(sorted_names[i])
                                continue

                            elif second_split == '3':
                                all_files[3].append(sorted_files[i])
                                all_names[3].append(sorted_names[i])
                                continue

                            elif second_split == '4':
                                all_files[4].append(sorted_files[i])
                                all_names[4].append(sorted_names[i])
                                continue

                    elif 'Campo' in folder or 'Claro' in folder:

                        print('Analizing brightfield picture folder')

                        os.chdir(folder)
                        files = filter(os.path.isfile, os.listdir(folder))
                        files = [os.path.join(folder, f) for f in files]
                        names = []
                        for file in files:

                            filename, ext = os.path.splitext(file)
                            #CHANGE THE EXTENSION TO TIFF OR TIF FOR IMAGE FILES
                            if ext == '.tif':
                                name = file.rsplit('/', 2)[2]
                                name = name.rsplit('.')[0]
                                names.append(name)
                            else:
                                continue

                        # We sort all files in the folder

                        sorted_files = Tcl().call('lsort', '-dict', files)
                        sorted_names = Tcl().call('lsort', '-dict', names)

                        for i in range(0, len(sorted_files)):

                            filename, ext = os.path.splitext(sorted_files[i])
                            first_split = filename.split(' ', 1)
                            second_split = first_split[1].split(' ')[1] # This gives us the specific day

                            print('File:', filename, '\nDay:', second_split)

                            if second_split == '0':
                                all_files[0].append(sorted_files[i])
                                all_names[0].append(sorted_names[i])
                                continue

                            elif second_split == '1':
                                all_files[1].append(sorted_files[i])
                                all_names[1].append(sorted_names[i])
                                continue

                            elif second_split == '2':
                                all_files[2].append(sorted_files[i])
                                all_names[2].append(sorted_names[i])
                                continue

                            elif second_split == '3':
                                all_files[3].append(sorted_files[i])
                                all_names[3].append(sorted_names[i])
                                continue

                            elif second_split == '4':
                                all_files[4].append(sorted_files[i])
                                all_names[4].append(sorted_names[i])
                                continue

            clear_output(wait=True) # Comment out this line if you would like to see the all the files detected
            
        clear_output(wait=True)
            
        return all_files, all_names

all_files, all_names = filesAndNames(5)

In [ ]:
'''
This block will create arrays for each of the plant types analyzed; the first array will contain
the full filepath of the image all_files and the second array will contain the filename of the image
'''

cam1 = []
cam1_names = []
ruby_thallus = []
ruby_thallus_names = []
ruby_thallus_fluo = []
ruby_thallus_fluo_names = []
ruby_spores = []
ruby_spores_names = []
ruby_spores_fluo = []
ruby_spores_fluo_names = []
hypet = []
hypet_names = []
hypet_fluo = []
hypet_fluo_names = []
venus_thallus = []
venus_thallus_names = []
venus_thallus_fluo = []
venus_thallus_fluo_names = []
venus_spores = []
venus_spores_names = []
venus_spores_fluo = []
venus_spores_fluo_names = []
venus_lti = []
venus_lti_names = []
venus_lti_fluo = []
venus_lti_fluo_names = []
venus_N7 = []
venus_N7_names = []
venus_N7_fluo = []
venus_N7_fluo_names = []
citrine = []
citrine_names = []
citrine_fluo = []
citrine_fluo_names = []
citrine_lti = []
citrine_lti_names = []
citrine_lti_fluo = []
citrine_lti_fluo_names = []
citrine_N7 = []
citrine_N7_names = []
citrine_N7_fluo = []
citrine_N7_fluo_names = []
turq = []
turq_names = []
turq_fluo = []
turq_fluo_names = []
turq_lti = []
turq_lti_names = []
turq_lti_fluo = []
turq_lti_fluo_names = []
turq_N7 = []
turq_N7_names = []
turq_N7_fluo = []
turq_N7_fluo_names = []
bfp_lti = []
bfp_lti_names = []
bfp_lti_fluo = []
bfp_lti_fluo_names = []
bfp_N7 = []
bfp_N7_names = []
bfp_N7_fluo = []
bfp_N7_fluo_names = []


for day in range(0, 5):

    cam1.append([file for file in all_files[day] if ('WT' or 'Wt') in file])
    cam1_names.append([name for name in all_names[day] if ('WT' or 'Wt') in name])
    ruby_thallus.append([file for file in all_files[day] if (('RUBY_T' in file) and ('F' not in file))])#la t siempre esta ...toma mas datos de los necesarios
    ruby_thallus_names.append([name for name in all_names[day] if (('RUBY_T' in name) and ('F' not in name))])
    ruby_thallus_fluo.append([file for file in all_files[day] if ('F RUBY_T' in file)])#lo mismo que antes...la t sobra
    ruby_thallus_fluo_names.append([name for name in all_names[day] if ('F RUBY_T' in name)])
    ruby_spores.append([file for file in all_files[day] if (('RUBY_E' in file) and ('F' not in file))]) #saque los filtros innecesarios deje solo la parte de ruby_e
    ruby_spores_names.append([name for name in all_names[day] if (('RUBY_E' in name) and ('F' not in name))])
    ruby_spores_fluo.append([file for file in all_files[day] if (('RUBY_E' in file) and ('F' in file))])
    ruby_spores_fluo_names.append([name for name in all_names[day] if (('RUBY_E' in name) and ('F' in name))])
    hypet.append([file for file in all_files[day] if (('YPETHEMME' in file) and ('F' not in file))])
    hypet_names.append([name for name in all_names[day] if (('YPETHEMME' in name) and ('F' not in name))])
    hypet_fluo.append([file for file in all_files[day] if ('F YPETHEMME' in file)])
    hypet_fluo_names.append([name for name in all_names[day] if ('F YPETHEMME' in name)])
    venus_thallus.append([file for file in all_files[day] if ('VENUS_T' in file and ('F' not in file))])
    venus_thallus_names.append([name for name in all_names[day] if (('VENUS_T' in name) and ('F' not in name))])
    venus_thallus_fluo.append([file for file in all_files[day] if ('F VENUS_T' in file)])
    venus_thallus_fluo_names.append([name for name in all_names[day] if ('F VENUS_T' in name)])
    venus_spores.append([file for file in all_files[day] if ('VENUS_E' in file and ('F' not in file))])
    venus_spores_names.append([name for name in all_names[day] if (('VENUS_E' in name) and ('F' not in name))])
    venus_spores_fluo.append([file for file in all_files[day] if ('F VENUS_E' in file)])
    venus_spores_fluo_names.append([name for name in all_names[day] if ('F VENUS_E' in name)])
    venus_lti.append([file for file in all_files[day] if ('VENUSLTI' in file and ('F' not in file))])
    venus_lti_names.append([name for name in all_names[day] if (('VENUSLTI' in name) and ('F' not in name))])
    venus_lti_fluo.append([file for file in all_files[day] if ('F VENUSLTI' in file)])
    venus_lti_fluo_names.append([name for name in all_names[day] if ('F VENUSLTI' in name)])
    venus_N7.append([file for file in all_files[day] if ('VENUSN7' in file and ('F' not in file))])
    venus_N7_names.append([name for name in all_names[day] if (('VENUSN7' in name) and ('F' not in name))])
    venus_N7_fluo.append([file for file in all_files[day] if ('F VENUSN7' in file)])
    venus_N7_fluo_names.append([name for name in all_names[day] if ('F VENUSN7' in name)])
    citrine.append([file for file in all_files[day] if (('CITRINE' in file) and ('F' not in file) and ('LTI' not in file) and ('N7' not in file))])
    citrine_names.append([name for name in all_names[day] if (('CITRINE' in name) and ('F' not in name) and ('LTI' not in name) and ('N7' not in name))])
    citrine_fluo.append([file for file in all_files[day] if (('F CITRINE' in file) and ('LTI' not in file) and ('N7' not in file))])
    citrine_fluo_names.append([name for name in all_names[day] if (('F CITRINE' in name) and ('LTI' not in name) and ('N7' not in name))])
    citrine_lti.append([file for file in all_files[day] if ('CITRINELTI' in file and ('F' not in file))])
    citrine_lti_names.append([name for name in all_names[day] if (('CITRINELTI' in name) and ('F' not in name))])
    citrine_lti_fluo.append([file for file in all_files[day] if ('F CITRINELTI' in file)])
    citrine_lti_fluo_names.append([name for name in all_names[day] if ('F CITRINELTI' in name)])
    citrine_N7.append([file for file in all_files[day] if ('CITRINEN7' in file and ('F' not in file))])
    citrine_N7_names.append([name for name in all_names[day] if (('CITRINEN7' in name) and ('F' not in name))])
    citrine_N7_fluo.append([file for file in all_files[day] if ('F CITRINEN7' in file)])
    citrine_N7_fluo_names.append([name for name in all_names[day] if ('F CITRINEN7' in name)])
    turq.append([file for file in all_files[day] if ('TURQ' in file and ('F' not in file) and ('LTI' not in file) and ('N7' not in file))])
    turq_names.append([name for name in all_names[day] if (('TURQ' in name) and ('F' not in name) and ('LTI' not in name) and ('N7' not in name))])
    turq_fluo.append([file for file in all_files[day] if ('F TURQ' in file and ('LTI' not in file) and ('N7' not in file))])
    turq_fluo_names.append([name for name in all_names[day] if (('F TURQ' in name) and ('LTI' not in name) and ('N7' not in name))])
    turq_lti.append([file for file in all_files[day] if ('TURQLTI' in file and ('F' not in file))])
    turq_lti_names.append([name for name in all_names[day] if (('TURQLTI' in name) and ('F' not in name))])
    turq_lti_fluo.append([file for file in all_files[day] if ('F TURQLTI' in file)])
    turq_lti_fluo_names.append([name for name in all_names[day] if ('F TURQLTI' in name)])
    turq_N7.append([file for file in all_files[day] if ('TURQN7' in file and ('F' not in file))])
    turq_N7_names.append([name for name in all_names[day] if (('TURQN7' in name) and ('F' not in name))])
    turq_N7_fluo.append([file for file in all_files[day] if ('F TURQN7' in file)])
    turq_N7_fluo_names.append([name for name in all_names[day] if ('F TURQN7' in name)])
    bfp_lti.append([file for file in all_files[day] if ('BFPLTI' in file) and ('F B' not in file)])
    bfp_lti_names.append([name for name in all_names[day] if ('BFPLTI' in name) and ('F B' not in name)])
    bfp_lti_fluo.append([file for file in all_files[day] if ('F BFPLTI' in file)])
    bfp_lti_fluo_names.append([name for name in all_names[day] if ('F BFPLTI' in name)])
    bfp_N7.append([file for file in all_files[day] if ('BFPN7' in file) and ('F B' not in file)])
    bfp_N7_names.append([name for name in all_names[day] if ('BFPN7' in name) and ('F B' not in name)])
    bfp_N7_fluo.append([file for file in all_files[day] if ('F BFPN7' in file)])
    bfp_N7_fluo_names.append([name for name in all_names[day] if ('F BFPN7' in name)])

In [ ]:
# We now define all the functions we will use for the analysis

def openImage(file):
    
    '''
    Open the TIF or TIFF file as a three dimensional array for further manipulation
    Input:
        - file: string which corresponds to the filepath of the image
    Output:
        - img: three dimensional array that contains all the image information
    '''
    
    img = io.imread(file)
    
    return img

def plotImage(image, index, day, plant):

    '''
    Plot a specific image, given a specific plant number, day and plant type
    Input:
        - image: (Array) Three dimensional array containing the RGB image information.
        - index: (Int) Integer number, has to be a number between the plant number 1 and the plant number N, where N is the number of total plants of a specific type.
        - day: (Int) Integer number, has to be a number between Day 0 and Day N, where N is the total number of days the experiment ran.
        - plant: (Str) String corresponding to the plant type, for example: BFPN7, BFPLTI, VENUSN7, etc.
    Output:
        - Matplotlib figure showing the image requested
    '''
    
    specific_names = [name for name in all_names[day] if plant in name or plant.upper() in name or plant.lower() in name]
    
    plt.figure(figsize=(5,5), dpi=200)
    plt.title(specific_names[index])
    plt.imshow(image, cmap='gray')

def exporsureAndContrast(image):

    '''
    Function used for Contrast Limited Adaptive Image Equalization (CLAHE).
    Inputs:
        - image: Three dimensional (RGB) or One dimensional array (Greyscale) image information.
    Output:
        - clahe_img: Three dimensional (RGB) or One dimensional array (Greyscale) image, with an equalized histogram.
    '''
    
    clahe_img = exposure.equalize_adapthist(image, clip_limit=0.02)
    
    return clahe_img
    
def image2HSV(image):

    '''
    Function used for the conversion of RGB images to HSV color space (Hue Saturation Value)
    Input:
        - image: Three dimensional (RGB) array, containing image information. Array cannot be Greyscale or any other dimensions than RGB
    Output:
        - img_HSV: Three dimensional (HSV) array, containing image information in HSV color space
    '''
    
    img_HSV = rgb2hsv(image)
    
    return img_HSV

def firstSegmentation(image):

    '''
    Function used for segmentation of OOIs from a given image. It works
    by generating a threshold using the Otsu algorithm and the Value channel
    from the HSV image. Afterwards, it generates a binary image by applying
    the threshold value.
    Input:
        - image: Three dimensional HSV (optional RGB) image.
    Output:
        - binary: Binary thresholded image.
    '''

    thresh = filters.threshold_otsu(image[:,:,2])
    
    binary = image[:,:,2] < thresh-0.13
    
    return binary

def secondSegmentation(image):
    
    '''
    Function used for segmentation of OOIs from a given image. It works
    by generating a threshold using the Otsu algorithm and the Value channel
    from the HSV image. Afterwards, it generates a binary image by applying
    the threshold value.
    Input:
        - image: Three dimensional HSV (optional RGB) image.
    Output:
        - binary: Binary thresholded image.
    '''
    
    thresh = filters.threshold_otsu(image)
    
    binary = image > thresh
    
    return binary

def objectProcessing(image):
    
    '''
    Main function used for the removal of unwanted objects and enhacement of
    OOIs. It works by a series of morphological binary operations, consisting of
    binary closing, object removal, hole filling and erosion.
    Input:
        - image: Input image must be a binary image, RGB and Greyscale images will NOT work.
    Output:
        - img_eroded: Processed binary image, containing the segmented OOIs.
    '''
    
    img_closing = morphology.binary_closing(image, selem=np.ones((30,30)))
    
    img_wo = morphology.remove_small_objects(img_closing, min_size=10000)
    
    img_closing2 = morphology.binary_closing(img_wo, selem=np.ones((15,15)))
    
    img_fill = ndi.binary_fill_holes(img_closing2)
    
    img_eroded = morphology.binary_erosion(img_fill, selem=np.ones((5,5)))
    
    return img_eroded

def area_count(binary_img):
    
    '''
    Function used for calculating the area of a segmented object in a binary image. Methods just loops all pixels
    and counts a pixel if True (1).
    Input:
        - binary_img: Binary image that will be used for counting. Image MUST be binary, as the code is written
        as to loop over a 1D image.
    Output:
        - count: Number of pixels in the image that were True (1).
    '''
    
    count = 0
    
    for row in binary_img:
        
        for pixel in row:
        
            if pixel == False:

                continue

            else:

                count += 1
            
    return (count)

def analyzeCAM1():
    
    '''
    Used for brightfield analysis of CAM1 wildtype plants
    '''
    # Deleted global delcaration of variable; it's unecessary for this code
    cam1_area = [[] for i in range(0, len(cam1))]
    
    for i in range(len(cam1)):
        
        print('Analyzing DAY {} Photos'.format(i))
        
        for j in range(len(cam1[i])):
            
            # Check the progress of algorithm
            print('Analyzing brightfield WT CAM1 image', cam1_names[i][j], j+1, 'of', len(cam1[i]))
            # Read the image
            img = openImage(cam1[i][j])
            # Convert to HSV color space for better segmentation
            HSV = image2HSV(img)
            # Segmentation
            segmented = firstSegmentation(HSV)
            # Object Processing
            wo = objectProcessing(segmented)
            # Calculate the area
            area = area_count(wo)
            # Convert pixels to sq. micrometers and append to results array
            cam1_area[i].append(area*25)
            # Print calculated area just for checking
            print('The approximate area for Image', cam1_names[i][j], 'is', area*25, 'square micrometers')
        
    return cam1_area

def analyzeRubyBFThallus():
    
    '''
    Used for brightfield analysis of ruby_thallus plants
    '''
    # Deleted global delcaration of variable; it's unecessary for this code
    ruby_thallus_area = [[] for i in range(0, len(ruby_thallus))]
    
    for i in range(len(ruby_thallus)):
        
        print('Analyzing DAY {} Photos'.format(i))
        
        for j in range(len(ruby_thallus[i])):
            
            # Check the progress of algorithm
            print('Analyzing brightfield ruby thallus image', ruby_thallus_names[i][j], j+1, 'of', len(ruby_thallus[i]))
            # Read the image
            img = openImage(ruby_thallus[i][j])
            # Convert to HSV color space for better segmentation
            HSV = image2HSV(img)
            # Segmentation
            segmented = firstSegmentation(HSV)
            # Object Processing
            wo = objectProcessing(segmented)
            # Calculate the area
            area = area_count(wo)
            # Convert pixels to sq. micrometers and append to results array
            ruby_thallus_area[i].append(area*25)
            # Print calculated area just for checking
            print('The approximate area for Image', ruby_thallus_names[i][j], 'is', area*25, 'square micrometers')
        
    return ruby_thallus_area
        
def analyzeRubyFLUOThallus():

    '''
    Code to analyze ruby_thallus_fluo images
    '''
    # Discarded global variable statement; it was unecessary.
    ruby_thallus_intensity = [[] for i in range(0, len(ruby_thallus_fluo))]

    for i in range(0, len(ruby_thallus_fluo)):
        
        print('Analyzing DAY {} images'.format(i))
        
        for j in range(0, len(ruby_thallus_fluo[i])):
        
            print('Analyzing fluorescent RUBY thallus image', ruby_thallus_fluo_names[i][j], j+1, 'of', len(ruby_thallus_fluo[i]))

            img = openImage(ruby_thallus_fluo[i][j])
            # We will use only the red channel for the fluorescence information 
            red_channel = img[:,:,0]
            # Segmentation
            segmented = secondSegmentation(red_channel)
            # We perform object processing within the Red Channel
            wo = objectProcessing(segmented)
            # We apply the mask obtained to each of the channels (R G and B)
            r = img[:,:,0]*wo
            g = img[:,:,1]*wo
            b = img[:,:,2]*wo
            # Final segmented image stacked into RGB
            final_rgb = np.dstack((r,g,b))
            # For the intensity calculation we convert the RGB image to greyscale
            greyscale_final = rgb2gray(final_rgb)
            # We calculate the intensity by calculating the sum of the greyscale values of the image
            intensity = round(np.sum(greyscale_final), 2)
            # Append calculated intensity to the corresponding array and print the intensity (to check progress)
            ruby_thallus_intensity[i].append(intensity)
            print('The intensity of the fluorescence for image', ruby_thallus_fluo_names[i][j], 'is', intensity) 

    return ruby_thallus_intensity

def analyzeRubyBFSpore():

    '''
    Used for brightfield analysis of ruby_spores plants
    '''
    # Deleted global delcaration of variable; it's unecessary for this code
    ruby_spores_area = [[] for i in range(0, len(ruby_spores))]
    
    for i in range(len(ruby_spores)):
        
        print('Analyzing DAY {} Photos'.format(i))
        
        for j in range(len(ruby_spores[i])):
            
            # Check the progress of algorithm
            print('Analyzing brightfield ruby spore image', ruby_spores_names[i][j], j+1, 'of', len(ruby_spores[i]))
            # Read the image
            img = openImage(ruby_spores[i][j])
            # Convert to HSV color space for better segmentation
            HSV = image2HSV(img)
            # Segmentation
            segmented = firstSegmentation(HSV)
            # Object Processing
            wo = objectProcessing(segmented)
            # Calculate the area
            area = area_count(wo)
            # Convert pixels to sq. micrometers and append to results array
            ruby_spores_area[i].append(area*25)
            # Print calculated area just for checking
            print('The approximate area for Image', ruby_spores_names[i][j], 'is', area*25, 'square micrometers')
        
    return ruby_spores_area
        
def analyzeRubyFLUOSpore():

    '''
    Code to analyze ruby_spores_fluo images
    '''
    # Discarded global variable statement; it was unecessary.
    ruby_spores_intensity = [[] for i in range(0, len(ruby_spores_fluo))]

    for i in range(0, len(ruby_spores_fluo)):
        
        print('Analyzing DAY {} images'.format(i))
        
        for j in range(0, len(ruby_spores_fluo[i])):
        
            print('Analyzing fluorescent RUBY spore image', ruby_spores_fluo_names[i][j], j+1, 'of', len(ruby_spores_fluo[i]))

            img = openImage(ruby_spores_fluo[i][j])
            # We will use only the red channel for the fluorescence information 
            red_channel = img[:,:,0]
            # Segmentation
            segmented = secondSegmentation(red_channel)
            # We perform object processing within the Red Channel
            wo = objectProcessing(segmented)
            # We apply the mask obtained to each of the channels (R G and B)
            r = img[:,:,0]*wo
            g = img[:,:,1]*wo
            b = img[:,:,2]*wo
            # Final segmented image stacked into RGB
            final_rgb = np.dstack((r,g,b))
            # For the intensity calculation we convert the RGB image to greyscale
            greyscale_final = rgb2gray(final_rgb)
            # We calculate the intensity by calculating the sum of the greyscale values of the image
            intensity = round(np.sum(greyscale_final), 2)
            # Append calculated intensity to the corresponding array and print the intensity (to check progress)
            ruby_spores_intensity[i].append(intensity)
            print('The intensity of the fluorescence for image', ruby_spores_fluo_names[i][j], 'is', intensity) 

    return ruby_spores_intensity
    
def analyzeHYPETBF():
    
    '''
    Used for brightfield analysis of hypet plants
    '''
    # Deleted global delcaration of variable; it's unecessary for this code
    hypet_area = [[] for i in range(0, len(hypet))]
    
    for i in range(len(hypet)):
        
        print('Analyzing DAY {} Photos'.format(i))
        
        for j in range(len(hypet[i])):
            
            # Check the progress of algorithm
            print('Analyzing brightfield hypet image', hypet_names[i][j], j+1, 'of', len(hypet[i]))
            # Read the image
            img = openImage(hypet[i][j])
            # Convert to HSV color space for better segmentation
            HSV = image2HSV(img)
            # Segmentation
            segmented = firstSegmentation(HSV)
            # Object Processing
            wo = objectProcessing(segmented)
            # Calculate the area
            area = area_count(wo)
            # Convert pixels to sq. micrometers and append to results array
            hypet_area[i].append(area*25)
            # Print calculated area just for checking
            print('The approximate area for Image', hypet_names[i][j], 'is', area*25, 'square micrometers')
        
    return hypet_area

def analyzeHYPETFLUO():

    '''
    Code to analyze hypet_fluo images
    '''
    # Discarded global variable stahypettement; it was unecessary.
    hypet_intensity = [[] for i in range(0, len(hypet_fluo))]

    for i in range(0, len(hypet_fluo)):
        
        print('Analyzing DAY {} images'.format(i))
        
        for j in range(0, len(hypet_fluo[i])):
        
            print('Analyzing fluorescent HYPET image', hypet_fluo_names[i][j], j+1, 'of', len(hypet_fluo[i]))

            img = openImage(hypet_fluo[i][j])
            # We will use only the green channel for the fluorescence information 
            green_channel = img[:,:,1]
            # Segmentation
            segmented = secondSegmentation(green_channel)
            # We perform object processing within the Green Channel
            wo = objectProcessing(segmented)
            # We apply the mask obtained to each of the channels (R G and B)
            r = img[:,:,0]*wo
            g = img[:,:,1]*wo
            b = img[:,:,2]*wo
            # Final segmented image stacked into RGB
            final_rgb = np.dstack((r,g,b))
            # For the intensity calculation we convert the RGB image to greyscale
            greyscale_final = rgb2gray(final_rgb)
            # We calculate the intensity by calculating the sum of the greyscale values of the image
            intensity = round(np.sum(greyscale_final), 2)
            # Append calculated intensity to the corresponding array and print the intensity (to check progress)
            hypet_intensity[i].append(intensity)
            print('The intensity of the fluorescence for image', hypet_fluo_names[i][j], 'is', intensity) 

    return hypet_intensity
        
def analyzeVenusBFThallus():

    '''
    Used for brightfield analysis of venus_thallus plants
    '''
    # Deleted global delcaration of variable; it's unecessary for this code
    venus_thallus_area = [[] for i in range(0, len(venus_thallus))]
    
    for i in range(len(venus_thallus)):
        
        print('Analyzing DAY {} Photos'.format(i))
        
        for j in range(len(venus_thallus[i])):
            
            # Check the progress of algorithm
            print('Analyzing brightfield venus thallus image', venus_thallus_names[i][j], j+1, 'of', len(venus_thallus[i]))
            # Read the image
            img = openImage(venus_thallus[i][j])
            # Convert to HSV color space for better segmentation
            HSV = image2HSV(img)
            # Segmentation
            segmented = firstSegmentation(HSV)
            # Object Processing
            wo = objectProcessing(segmented)
            # Calculate the area
            area = area_count(wo)
            # Convert pixels to sq. micrometers and append to results array
            venus_thallus_area[i].append(area*25)
            # Print calculated area just for checking
            print('The approximate area for Image', venus_thallus_names[i][j], 'is', area*25, 'square micrometers')
        
    return venus_thallus_area

def analyzeVenusFluoThallus():

    '''
    Code to analyze venus_thallus_fluo images
    '''
    # Discarded global variable stavenus_thallustement; it was unecessary.
    venus_thallus_intensity = [[] for i in range(0, len(venus_thallus_fluo))]

    for i in range(0, len(venus_thallus_fluo)):
        
        print('Analyzing DAY {} images'.format(i))
        
        for j in range(0, len(venus_thallus_fluo[i])):
        
            print('Analyzing fluorescent venus thallus image', venus_thallus_fluo_names[i][j], j+1, 'of', len(venus_thallus_fluo[i]))

            img = openImage(venus_thallus_fluo[i][j])
            # We will use only the green channel for the fluorescence information 
            green_channel = img[:,:,1]
            # Segmentation
            segmented = secondSegmentation(green_channel)
            # We perform object processing within the Green Channel
            wo = objectProcessing(segmented)
            # We apply the mask obtained to each of the channels (R G and B)
            r = img[:,:,0]*wo
            g = img[:,:,1]*wo
            b = img[:,:,2]*wo
            # Final segmented image stacked into RGB
            final_rgb = np.dstack((r,g,b))
            # For the intensity calculation we convert the RGB image to greyscale
            greyscale_final = rgb2gray(final_rgb)
            # We calculate the intensity by calculating the sum of the greyscale values of the image
            intensity = round(np.sum(greyscale_final), 2)
            # Append calculated intensity to the corresponding array and print the intensity (to check progress)
            venus_thallus_intensity[i].append(intensity)
            print('The intensity of the fluorescence for image', venus_thallus_fluo_names[i][j], 'is', intensity) 

    return venus_thallus_intensity

def analyzeVenusBFSpore():
    
    '''
    Used for brightfield analysis of venus_spores plants
    '''
    # Deleted global delcaration of variable; it's unecessary for this code
    venus_spores_area = [[] for i in range(0, len(venus_spores))]
    
    for i in range(len(venus_spores)):
        
        print('Analyzing DAY {} Photos'.format(i))
        
        for j in range(len(venus_spores[i])):
            
            # Check the progress of algorithm
            print('Analyzing brightfield venus spore image', venus_spores_names[i][j], j+1, 'of', len(venus_spores[i]))
            # Read the image
            img = openImage(venus_spores[i][j])
            # Convert to HSV color space for better segmentation
            HSV = image2HSV(img)
            # Segmentation
            segmented = firstSegmentation(HSV)
            # Object Processing
            wo = objectProcessing(segmented)
            # Calculate the area
            area = area_count(wo)
            # Convert pixels to sq. micrometers and append to results array
            venus_spores_area[i].append(area*25)
            # Print calculated area just for checking
            print('The approximate area for Image', venus_spores_names[i][j], 'is', area*25, 'square micrometers')
        
    return venus_spores_area

def analyzeVenusFluoSpore():
    
    '''
    Code to analyze venus_spores_fluo images
    '''
    # Discarded global variable stavenus_sporestement; it was unecessary.
    venus_spores_intensity = [[] for i in range(0, len(venus_spores_fluo))]

    for i in range(0, len(venus_spores_fluo)):
        
        print('Analyzing DAY {} images'.format(i))
        
        for j in range(0, len(venus_spores_fluo[i])):
        
            print('Analyzing fluorescent venus spore image', venus_spores_fluo_names[i][j], j+1, 'of', len(venus_spores_fluo[i]))

            img = openImage(venus_spores_fluo[i][j])
            # We will use only the green channel for the fluorescence information 
            green_channel = img[:,:,1]
            # Segmentation
            segmented = secondSegmentation(green_channel)
            # We perform object processing within the Green Channel
            wo = objectProcessing(segmented)
            # We apply the mask obtained to each of the channels (R G and B)
            r = img[:,:,0]*wo
            g = img[:,:,1]*wo
            b = img[:,:,2]*wo
            # Final segmented image stacked into RGB
            final_rgb = np.dstack((r,g,b))
            # For the intensity calculation we convert the RGB image to greyscale
            greyscale_final = rgb2gray(final_rgb)
            # We calculate the intensity by calculating the sum of the greyscale values of the image
            intensity = round(np.sum(greyscale_final), 2)
            # Append calculated intensity to the corresponding array and print the intensity (to check progress)
            venus_spores_intensity[i].append(intensity)
            print('The intensity of the fluorescence for image', venus_spores_fluo_names[i][j], 'is', intensity) 

    return venus_spores_intensity

def analyzeVenusLTIBF():

    '''
    Used for brightfield analysis of venus_lti plants
    '''
    # Deleted global delcaration of variable; it's unecessary for this code
    venus_lti_area = [[] for i in range(0, len(venus_lti))]
    
    for i in range(len(venus_lti)):
        
        print('Analyzing DAY {} Photos'.format(i))
        
        for j in range(len(venus_lti[i])):
            
            # Check the progress of algorithm
            print('Analyzing brightfield venus LTI image', venus_lti_names[i][j], j+1, 'of', len(venus_lti[i]))
            # Read the image
            img = openImage(venus_lti[i][j])
            # Convert to HSV color space for better segmentation
            HSV = image2HSV(img)
            # Segmentation
            segmented = firstSegmentation(HSV)
            # Object Processing
            wo = objectProcessing(segmented)
            # Calculate the area
            area = area_count(wo)
            # Convert pixels to sq. micrometers and append to results array
            venus_lti_area[i].append(area*25)
            # Print calculated area just for checking
            print('The approximate area for Image', venus_lti_names[i][j], 'is', area*25, 'square micrometers')
        
    return venus_lti_area

def analyzeVenusLTIFluo():
    
    '''
    Code to analyze venus_lti_fluo images
    '''
    # Discarded global variable stavenus_ltitement; it was unecessary.
    venus_lti_intensity = [[] for i in range(0, len(venus_lti_fluo))]

    for i in range(0, len(venus_lti_fluo)):
        
        print('Analyzing DAY {} images'.format(i))
        
        for j in range(0, len(venus_lti_fluo[i])):
        
            print('Analyzing fluorescent venus LTI image', venus_lti_fluo_names[i][j], j+1, 'of', len(venus_lti_fluo[i]))

            img = openImage(venus_lti_fluo[i][j])
            # We will use only the green channel for the fluorescence information 
            green_channel = img[:,:,1]
            # Segmentation
            segmented = secondSegmentation(green_channel)
            # We perform object processing within the Green Channel
            wo = objectProcessing(segmented)
            # We apply the mask obtained to each of the channels (R G and B)
            r = img[:,:,0]*wo
            g = img[:,:,1]*wo
            b = img[:,:,2]*wo
            # Final segmented image stacked into RGB
            final_rgb = np.dstack((r,g,b))
            # For the intensity calculation we convert the RGB image to greyscale
            greyscale_final = rgb2gray(final_rgb)
            # We calculate the intensity by calculating the sum of the greyscale values of the image
            intensity = round(np.sum(greyscale_final), 2)
            # Append calculated intensity to the corresponding array and print the intensity (to check progress)
            venus_lti_intensity[i].append(intensity)
            print('The intensity of the fluorescence for image', venus_lti_fluo_names[i][j], 'is', intensity) 

    return venus_lti_intensity
    
def analyzeVenusN7BF():

    '''
    Used for brightfield analysis of venus_N7 plants
    '''
    # Deleted global delcaration of variable; it's unecessary for this code
    venus_N7_area = [[] for i in range(0, len(venus_N7))]
    
    for i in range(len(venus_N7)):
        
        print('Analyzing DAY {} Photos'.format(i))
        
        for j in range(len(venus_N7[i])):
            
            # Check the progress of algorithm
            print('Analyzing brightfield venus N7 image', venus_N7_names[i][j], j+1, 'of', len(venus_N7[i]))
            # Read the image
            img = openImage(venus_N7[i][j])
            # Convert to HSV color space for better segmentation
            HSV = image2HSV(img)
            # Segmentation
            segmented = firstSegmentation(HSV)
            # Object Processing
            wo = objectProcessing(segmented)
            # Calculate the area
            area = area_count(wo)
            # Convert pixels to sq. micrometers and append to results array
            venus_N7_area[i].append(area*25)
            # Print calculated area just for checking
            print('The approximate area for Image', venus_N7_names[i][j], 'is', area*25, 'square micrometers')
        
    return venus_N7_area

def analyzeVenusN7Fluo():
    
    '''
    Code to analyze venus_N7_fluo images
    '''
    # Discarded global variable stavenus_N7tement; it was unecessary.
    venus_N7_intensity = [[] for i in range(0, len(venus_N7_fluo))]

    for i in range(0, len(venus_N7_fluo)):
        
        print('Analyzing DAY {} images'.format(i))
        
        for j in range(0, len(venus_N7_fluo[i])):
        
            print('Analyzing fluorescent venus N7 image', venus_N7_fluo_names[i][j], j+1, 'of', len(venus_N7_fluo[i]))

            img = openImage(venus_N7_fluo[i][j])
            # We will use only the green channel for the fluorescence information 
            green_channel = np.multiply(img[:,:,1], 1.5)
            # Segmentation
            segmented = secondSegmentation(green_channel)
            # We perform object processing within the Green Channel
            wo = objectProcessing(segmented)
            # We apply the mask obtained to each of the channels (R G and B)
            r = img[:,:,0]*wo
            g = img[:,:,1]*wo
            b = img[:,:,2]*wo
            # Final segmented image stacked into RGB
            final_rgb = np.dstack((r,g,b))
            # For the intensity calculation we convert the RGB image to greyscale
            greyscale_final = rgb2gray(final_rgb)
            # We calculate the intensity by calculating the sum of the greyscale values of the image
            intensity = round(np.sum(greyscale_final), 2)
            # Append calculated intensity to the corresponding array and print the intensity (to check progress)
            venus_N7_intensity[i].append(intensity)
            print('The intensity of the fluorescence for image', venus_N7_fluo_names[i][j], 'is', intensity) 

    return venus_N7_intensity

def analyzeCitrineBF():

    '''
    Used for brightfield analysis of citrine plants
    '''
    # Deleted global delcaration of variable; it's unecessary for this code
    citrine_area = [[] for i in range(0, len(citrine))]
    
    for i in range(len(citrine)):
        
        print('Analyzing DAY {} Photos'.format(i))
        
        for j in range(len(citrine[i])):
            
            # Check the progress of algorithm
            print('Analyzing brightfield citrine image', citrine_names[i][j], j+1, 'of', len(citrine[i]))
            # Read the image
            img = openImage(citrine[i][j])
            # Convert to HSV color space for better segmentation
            HSV = image2HSV(img)
            # Segmentation
            segmented = firstSegmentation(HSV)
            # Object Processing
            wo = objectProcessing(segmented)
            # Calculate the area
            area = area_count(wo)
            # Convert pixels to sq. micrometers and append to results array
            citrine_area[i].append(area*25)
            # Print calculated area just for checking
            print('The approximate area for Image', citrine_names[i][j], 'is', area*25, 'square micrometers')
        
    return citrine_area

def analyzeCitrineFluo():
    
    '''
    Code to analyze citrine_fluo images
    '''
    # Discarded global variable stacitrinetement; it was unecessary.
    citrine_intensity = [[] for i in range(0, len(citrine_fluo))]

    for i in range(0, len(citrine_fluo)):
        
        print('Analyzing DAY {} images'.format(i))
        
        for j in range(0, len(citrine_fluo[i])):
        
            print('Analyzing fluorescent citrine image', citrine_fluo_names[i][j], j+1, 'of', len(citrine_fluo[i]))

            img = openImage(citrine_fluo[i][j])
            # We will use only the green channel for the fluorescence information 
            green_channel = img[:,:,1]
            # Segmentation
            segmented = secondSegmentation(green_channel)
            # We perform object processing within the Green Channel
            wo = objectProcessing(segmented)
            # We apply the mask obtained to each of the channels (R G and B)
            r = img[:,:,0]*wo
            g = img[:,:,1]*wo
            b = img[:,:,2]*wo
            # Final segmented image stacked into RGB
            final_rgb = np.dstack((r,g,b))
            # For the intensity calculation we convert the RGB image to greyscale
            greyscale_final = rgb2gray(final_rgb)
            # We calculate the intensity by calculating the sum of the greyscale values of the image
            intensity = round(np.sum(greyscale_final), 2)
            # Append calculated intensity to the corresponding array and print the intensity (to check progress)
            citrine_intensity[i].append(intensity)
            print('The intensity of the fluorescence for image', citrine_fluo_names[i][j], 'is', intensity) 

    return citrine_intensity

def analyzeCitrineLTIBF():

    '''
    Used for brightfield analysis of citrine_lti plants
    '''
    # Deleted global delcaration of variable; it's unecessary for this code
    citrine_lti_area = [[] for i in range(0, len(citrine_lti))]
    
    for i in range(len(citrine_lti)):
        
        print('Analyzing DAY {} Photos'.format(i))
        
        for j in range(len(citrine_lti[i])):
            
            # Check the progress of algorithm
            print('Analyzing brightfield citrine LTI image', citrine_lti_names[i][j], j+1, 'of', len(citrine_lti[i]))
            # Read the image
            img = openImage(citrine_lti[i][j])
            # Convert to HSV color space for better segmentation
            HSV = image2HSV(img)
            # Segmentation
            segmented = firstSegmentation(HSV)
            # Object Processing
            wo = objectProcessing(segmented)
            # Calculate the area
            area = area_count(wo)
            # Convert pixels to sq. micrometers and append to results array
            citrine_lti_area[i].append(area*25)
            # Print calculated area just for checking
            print('The approximate area for Image', citrine_lti_names[i][j], 'is', area*25, 'square micrometers')
        
    return citrine_lti_area

def analyzeCitrineLTIFluo():
    
    '''
    Code to analyze citrine_lti_fluo images
    '''
    # Discarded global variable stacitrine_ltitement; it was unecessary.
    citrine_lti_intensity = [[] for i in range(0, len(citrine_lti_fluo))]

    for i in range(0, len(citrine_lti_fluo)):
        
        print('Analyzing DAY {} images'.format(i))
        
        for j in range(0, len(citrine_lti_fluo[i])):
        
            print('Analyzing fluorescent citrine LTI image', citrine_lti_fluo_names[i][j], j+1, 'of', len(citrine_lti_fluo[i]))

            img = openImage(citrine_lti_fluo[i][j])
            # We will use only the green channel for the fluorescence information 
            green_channel = img[:,:,1]
            # Segmentation
            segmented = secondSegmentation(green_channel)
            # We perform object processing within the Green Channel
            wo = objectProcessing(segmented)
            # We apply the mask obtained to each of the channels (R G and B)
            r = img[:,:,0]*wo
            g = img[:,:,1]*wo
            b = img[:,:,2]*wo
            # Final segmented image stacked into RGB
            final_rgb = np.dstack((r,g,b))
            # For the intensity calculation we convert the RGB image to greyscale
            greyscale_final = rgb2gray(final_rgb)
            # We calculate the intensity by calculating the sum of the greyscale values of the image
            intensity = round(np.sum(greyscale_final), 2)
            # Append calculated intensity to the corresponding array and print the intensity (to check progress)
            citrine_lti_intensity[i].append(intensity)
            print('The intensity of the fluorescence for image', citrine_lti_fluo_names[i][j], 'is', intensity) 

    return citrine_lti_intensity

def analyzeCitrineN7BF():

    '''
    Used for brightfield analysis of citrine_N7 plants
    '''
    # Deleted global delcaration of variable; it's unecessary for this code
    citrine_N7_area = [[] for i in range(0, len(citrine_N7))]
    
    for i in range(len(citrine_N7)):
        
        print('Analyzing DAY {} Photos'.format(i))
        
        for j in range(len(citrine_N7[i])):
            
            # Check the progress of algorithm
            print('Analyzing brightfield citrine N7 image', citrine_N7_names[i][j], j+1, 'of', len(citrine_N7[i]))
            # Read the image
            img = openImage(citrine_N7[i][j])
            # Convert to HSV color space for better segmentation
            HSV = image2HSV(img)
            # Segmentation
            segmented = firstSegmentation(HSV)
            # Object Processing
            wo = objectProcessing(segmented)
            # Calculate the area
            area = area_count(wo)
            # Convert pixels to sq. micrometers and append to results array
            citrine_N7_area[i].append(area*25)
            # Print calculated area just for checking
            print('The approximate area for Image', citrine_N7_names[i][j], 'is', area*25, 'square micrometers')
        
    return citrine_N7_area

def analyzeCitrineN7Fluo():
    '''
    Code to analyze citrine_N7_fluo images
    '''
    # Discarded global variable stacitrine_N7tement; it was unecessary.
    citrine_N7_intensity = [[] for i in range(0, len(citrine_N7_fluo))]

    for i in range(0, len(citrine_N7_fluo)):
        
        print('Analyzing DAY {} images'.format(i))
        
        for j in range(0, len(citrine_N7_fluo[i])):
        
            print('Analyzing fluorescent citrine N7 image', citrine_N7_fluo_names[i][j], j+1, 'of', len(citrine_N7_fluo[i]))

            img = openImage(citrine_N7_fluo[i][j])
            # We will use only the green channel for the fluorescence information 
            green_channel = np.multiply(img[:,:,1], 1.5)
            # Segmentation
            segmented = secondSegmentation(green_channel)
            # We perform object processing within the Green Channel
            wo = objectProcessing(segmented)
            # We apply the mask obtained to each of the channels (R G and B)
            r = img[:,:,0]*wo
            g = img[:,:,1]*wo
            b = img[:,:,2]*wo
            # Final segmented image stacked into RGB
            final_rgb = np.dstack((r,g,b))
            # For the intensity calculation we convert the RGB image to greyscale
            greyscale_final = rgb2gray(final_rgb)
            # We calculate the intensity by calculating the sum of the greyscale values of the image
            intensity = round(np.sum(greyscale_final), 2)
            # Append calculated intensity to the corresponding array and print the intensity (to check progress)
            citrine_N7_intensity[i].append(intensity)
            print('The intensity of the fluorescence for image', citrine_N7_fluo_names[i][j], 'is', intensity) 

    return citrine_N7_intensity

def analyzeTurquoiseBF():

    '''
    Used for brightfield analysis of turq plants
    '''
    # Deleted global delcaration of variable; it's unecessary for this code
    turq_area = [[] for i in range(0, len(turq))]
    
    for i in range(len(turq)):
        
        print('Analyzing DAY {} Photos'.format(i))
        
        for j in range(len(turq[i])):
            
            # Check the progress of algorithm
            print('Analyzing brightfield turquoise image', turq_names[i][j], j+1, 'of', len(turq[i]))
            # Read the image
            img = openImage(turq[i][j])
            # Convert to HSV color space for better segmentation
            HSV = image2HSV(img)
            # Segmentation
            segmented = firstSegmentation(HSV)
            # Object Processing
            wo = objectProcessing(segmented)
            # Calculate the area
            area = area_count(wo)
            # Convert pixels to sq. micrometers and append to results array
            turq_area[i].append(area*25)
            # Print calculated area just for checking
            print('The approximate area for Image', turq_names[i][j], 'is', area*25, 'square micrometers')
        
    return turq_area

def analyzeTurquoiseFluo():
    
    '''
    Code to analyze turq_fluo images
    '''
    # Discarded global variable staturqtement; it was unecessary.
    turq_intensity = [[] for i in range(0, len(turq_fluo))]

    for i in range(0, len(turq_fluo)):
        
        print('Analyzing DAY {} images'.format(i))
        
        for j in range(0, len(turq_fluo[i])):
        
            print('Analyzing fluorescent turquoise image', turq_fluo_names[i][j], j+1, 'of', len(turq_fluo[i]))

            img = openImage(turq_fluo[i][j])
            # We will use only the blue channel for the fluorescence information 
            blue_channel = np.multiply(img[:,:,2], 1.5)
            # Segmentation
            segmented = secondSegmentation(blue_channel)
            # We perform object processing within the Blue Channel
            wo = objectProcessing(segmented)
            # We apply the mask obtained to each of the channels (R G and B)
            r = img[:,:,0]*wo
            g = img[:,:,1]*wo
            b = img[:,:,2]*wo
            # Final segmented image stacked into RGB
            final_rgb = np.dstack((r,g,b))
            # For the intensity calculation we convert the RGB image to greyscale
            greyscale_final = rgb2gray(final_rgb)
            # We calculate the intensity by calculating the sum of the greyscale values of the image
            intensity = round(np.sum(greyscale_final), 2)
            # Append calculated intensity to the corresponding array and print the intensity (to check progress)
            turq_intensity[i].append(intensity)
            print('The intensity of the fluorescence for image', turq_fluo_names[i][j], 'is', intensity) 

    return turq_intensity

def analyzeTurquoiseLTIBF():
    
    '''
    Used for brightfield analysis of turq_lti plants
    '''
    # Deleted global delcaration of variable; it's unecessary for this code
    turq_lti_area = [[] for i in range(0, len(turq_lti))]
    
    for i in range(len(turq_lti)):
        
        print('Analyzing DAY {} Photos'.format(i))
        
        for j in range(len(turq_lti[i])):
            
            # Check the progress of algorithm
            print('Analyzing brightfield turquoise LTI image', turq_lti_names[i][j], j+1, 'of', len(turq_lti[i]))
            # Read the image
            img = openImage(turq_lti[i][j])
            # Convert to HSV color space for better segmentation
            HSV = image2HSV(img)
            # Segmentation
            segmented = firstSegmentation(HSV)
            # Object Processing
            wo = objectProcessing(segmented)
            # Calculate the area
            area = area_count(wo)
            # Convert pixels to sq. micrometers and append to results array
            turq_lti_area[i].append(area*25)
            # Print calculated area just for checking
            print('The approximate area for Image', turq_lti_names[i][j], 'is', area*25, 'square micrometers')
        
    return turq_lti_area

def analyzeTurquoiseLTIFluo():

    '''
    Code to analyze turq_lti_fluo images
    '''
    # Discarded global variable staturq_ltitement; it was unecessary.
    turq_lti_intensity = [[] for i in range(0, len(turq_lti_fluo))]

    for i in range(0, len(turq_lti_fluo)):
        
        print('Analyzing DAY {} images'.format(i))
        
        for j in range(0, len(turq_lti_fluo[i])):
        
            print('Analyzing fluorescent turquoise LTI image', turq_lti_fluo_names[i][j], j+1, 'of', len(turq_lti_fluo[i]))

            img = openImage(turq_lti_fluo[i][j])
            # We will use only the blue channel for the fluorescence information 
            blue_channel = np.multiply(img[:,:,2], 1.5)
            # Segmentation
            segmented = secondSegmentation(blue_channel)
            # We perform object processing within the Blue Channel
            wo = objectProcessing(segmented)
            # We apply the mask obtained to each of the channels (R G and B)
            r = img[:,:,0]*wo
            g = img[:,:,1]*wo
            b = img[:,:,2]*wo
            # Final segmented image stacked into RGB
            final_rgb = np.dstack((r,g,b))
            # For the intensity calculation we convert the RGB image to greyscale
            greyscale_final = rgb2gray(final_rgb)
            # We calculate the intensity by calculating the sum of the greyscale values of the image
            intensity = round(np.sum(greyscale_final), 2)
            # Append calculated intensity to the corresponding array and print the intensity (to check progress)
            turq_lti_intensity[i].append(intensity)
            print('The intensity of the fluorescence for image', turq_lti_fluo_names[i][j], 'is', intensity) 

    return turq_lti_intensity

def analyzeTurquoiseN7BF():

    '''
    Used for brightfield analysis of turq_N7 plants
    '''
    # Deleted global delcaration of variable; it's unecessary for this code
    turq_N7_area = [[] for i in range(0, len(turq_N7))]
    
    for i in range(len(turq_N7)):
        
        print('Analyzing DAY {} Photos'.format(i))
        
        for j in range(len(turq_N7[i])):
            
            # Check the progress of algorithm
            print('Analyzing brightfield turquoise N7 image', turq_N7_names[i][j], j+1, 'of', len(turq_N7[i]))
            # Read the image
            img = openImage(turq_N7[i][j])
            # Convert to HSV color space for better segmentation
            HSV = image2HSV(img)
            # Segmentation
            segmented = firstSegmentation(HSV)
            # Object Processing
            wo = objectProcessing(segmented)
            # Calculate the area
            area = area_count(wo)
            # Convert pixels to sq. micrometers and append to results array
            turq_N7_area[i].append(area*25)
            # Print calculated area just for checking
            print('The approximate area for Image', turq_N7_names[i][j], 'is', area*25, 'square micrometers')
        
    return turq_N7_area

def analyzeTurquoiseN7Fluo():
    
    '''
    Code to analyze turq_N7_fluo images
    '''
    # Discarded global variable staturq_N7tement; it was unecessary.
    turq_N7_intensity = [[] for i in range(0, len(turq_N7_fluo))]

    for i in range(0, len(turq_N7_fluo)):
        
        print('Analyzing DAY {} images'.format(i))
        
        for j in range(0, len(turq_N7_fluo[i])):
        
            print('Analyzing fluorescent turquoise N7 image', turq_N7_fluo_names[i][j], j+1, 'of', len(turq_N7_fluo[i]))

            img = openImage(turq_N7_fluo[i][j])
            # We will use only the blue channel for the fluorescence information 
            blue_channel = np.multiply(img[:,:,2], 2.5)
            # Segmentation
            segmented = secondSegmentation(blue_channel)
            # We perform object processing within the Blue Channel
            wo = objectProcessing(segmented)
            # We apply the mask obtained to each of the channels (R G and B)
            r = img[:,:,0]*wo
            g = img[:,:,1]*wo
            b = img[:,:,2]*wo
            # Final segmented image stacked into RGB
            final_rgb = np.dstack((r,g,b))
            # For the intensity calculation we convert the RGB image to greyscale
            greyscale_final = rgb2gray(final_rgb)
            # We calculate the intensity by calculating the sum of the greyscale values of the image
            intensity = round(np.sum(greyscale_final), 2)
            # Append calculated intensity to the corresponding array and print the intensity (to check progress)
            turq_N7_intensity[i].append(intensity)
            print('The intensity of the fluorescence for image', turq_N7_fluo_names[i][j], 'is', intensity) 

    return turq_N7_intensity

def analyzeBFPLTIBF():

    '''
    Used for brightfield analysis of bfp_lti plants
    '''
    # Deleted global delcaration of variable; it's unecessary for this code
    bfp_lti_area = [[] for i in range(0, len(bfp_lti))]
    
    for i in range(len(bfp_lti)):
        
        print('Analyzing DAY {} Photos'.format(i))
        
        for j in range(len(bfp_lti[i])):
            
            # Check the progress of algorithm
            print('Analyzing brightfield BFP LTI image', bfp_lti_names[i][j], j+1, 'of', len(bfp_lti[i]))
            # Read the image
            img = openImage(bfp_lti[i][j])
            # Convert to HSV color space for better segmentation
            HSV = image2HSV(img)
            # Segmentation
            segmented = firstSegmentation(HSV)
            # Object Processing
            wo = objectProcessing(segmented)
            # Calculate the area
            area = area_count(wo)
            # Convert pixels to sq. micrometers and append to results array
            bfp_lti_area[i].append(area*25)
            # Print calculated area just for checking
            print('The approximate area for Image', bfp_lti_names[i][j], 'is', area*25, 'square micrometers')
        
    return bfp_lti_area

def analyzeBFPLTIFluo():
    
    '''
    Code to analyze bfp_lti_fluo images
    '''
    # Discarded global variable stabfp_ltitement; it was unecessary.
    bfp_lti_intensity = [[] for i in range(0, len(bfp_lti_fluo))]

    for i in range(0, len(bfp_lti_fluo)):
        
        print('Analyzing DAY {} images'.format(i))
        
        for j in range(0, len(bfp_lti_fluo[i])):
        
            print('Analyzing fluorescent BFP LTI image', bfp_lti_fluo_names[i][j], j+1, 'of', len(bfp_lti_fluo[i]))

            img = openImage(bfp_lti_fluo[i][j])
            # We will use only the blue channel for the fluorescence information 
            blue_channel = np.multiply(img[:,:,2], 1.75)
            # Segmentation
            segmented = secondSegmentation(blue_channel)
            # We perform object processing within the Blue Channel
            wo = objectProcessing(segmented)
            # We apply the mask obtained to each of the channels (R G and B)
            r = img[:,:,0]*wo
            g = img[:,:,1]*wo
            b = img[:,:,2]*wo
            # Final segmented image stacked into RGB
            final_rgb = np.dstack((r,g,b))
            # For the intensity calculation we convert the RGB image to greyscale
            greyscale_final = rgb2gray(final_rgb)
            # We calculate the intensity by calculating the sum of the greyscale values of the image
            intensity = round(np.sum(greyscale_final), 2)
            # Append calculated intensity to the corresponding array and print the intensity (to check progress)
            bfp_lti_intensity[i].append(intensity)
            print('The intensity of the fluorescence for image', bfp_lti_fluo_names[i][j], 'is', intensity) 

    return bfp_lti_intensity

def analyzeBFPN7BF():

    '''
    Used for brightfield analysis of bfp_N7 plants
    '''
    # Deleted global delcaration of variable; it's unecessary for this code
    bfp_N7_area = [[] for i in range(0, len(bfp_N7))]
    
    for i in range(len(bfp_N7)):
        
        print('Analyzing DAY {} Photos'.format(i))
        
        for j in range(len(bfp_N7[i])):
            
            # Check the progress of algorithm
            print('Analyzing brightfield BFP N7 image', bfp_N7_names[i][j], j+1, 'of', len(bfp_N7[i]))
            # Read the image
            img = openImage(bfp_N7[i][j])
            # Convert to HSV color space for better segmentation
            HSV = image2HSV(img)
            # Segmentation
            segmented = firstSegmentation(HSV)
            # Object Processing
            wo = objectProcessing(segmented)
            # Calculate the area
            area = area_count(wo)
            # Convert pixels to sq. micrometers and append to results array
            bfp_N7_area[i].append(area*25)
            # Print calculated area just for checking
            print('The approximate area for Image', bfp_N7_names[i][j], 'is', area*25, 'square micrometers')
        
    return bfp_N7_area

def analyzeBFPN7Fluo():
    
    '''
    Code to analyze bfp_N7_fluo images
    '''
    # Discarded global variable stabfp_N7tement; it was unecessary.
    bfp_N7_intensity = [[] for i in range(0, len(bfp_N7_fluo))]

    for i in range(0, len(bfp_N7_fluo)):
        
        print('Analyzing DAY {} images'.format(i))
        
        for j in range(0, len(bfp_N7_fluo[i])):
        
            print('Analyzing fluorescent BFP N7 image', bfp_N7_fluo_names[i][j], j+1, 'of', len(bfp_N7_fluo[i]))

            img = openImage(bfp_N7_fluo[i][j])
            # We will use only the blue channel for the fluorescence information 
            blue_channel = np.multiply(img[:,:,2], 1.75)
            # Segmentation
            segmented = secondSegmentation(blue_channel)
            # We perform object processing within the Blue Channel
            wo = objectProcessing(segmented)
            # We apply the mask obtained to each of the channels (R G and B)
            r = img[:,:,0]*wo
            g = img[:,:,1]*wo
            b = img[:,:,2]*wo
            # Final segmented image stacked into RGB
            final_rgb = np.dstack((r,g,b))
            # For the intensity calculation we convert the RGB image to greyscale
            greyscale_final = rgb2gray(final_rgb)
            # We calculate the intensity by calculating the sum of the greyscale values of the image
            intensity = round(np.sum(greyscale_final), 2)
            # Append calculated intensity to the corresponding array and print the intensity (to check progress)
            bfp_N7_intensity[i].append(intensity)
            print('The intensity of the fluorescence for image', bfp_N7_fluo_names[i][j], 'is', intensity) 

    return bfp_N7_intensity

## Output from analyze functions

The output produced from the analyze functions is an array of arrays; To clarify, the length of the main array is the number of days that were used for taking images. Within each of these subarrays (each of the days) is the data for each of the plants.

| Area/Day | Day 0 | Day 1 | Day 2 | Day 3 | Day 4 |
| :---: | :---: | :---: | :---: | :---: | :---: |
| Cam1 Area | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] |
| HYPET Area | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] |
| Ruby Area Spore | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] |
| Ruby Area Thallus | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] |
| Venus Area Spore | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] |
| Venus Area Thallus | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] |
| Venus LTI Area | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] |
| Venus N7 Area | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] |
| Citrine Area | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] |
| Citrine LTI Area | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] |
| Citrine N7 Area | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] |
| Turquoise Area | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] |
| Turquoise LTI Area | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] |
| Turquoise N7 Area | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] |
| BFP LTI Area | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] |
| BFP N7 Area | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] | [Plant 1, Plant 2, ... , Plant n] |

In [ ]:
'''
For the statistical analysis and the graphs we will store all calculated areas and intensities on the
following dictionaries: all_areas and all_intensities
'''

brightfield_functions = [analyzeCAM1, analyzeRubyBFThallus, analyzeRubyBFSpore, analyzeHYPETBF, analyzeVenusBFThallus, analyzeVenusBFSpore, analyzeVenusLTIBF, analyzeVenusN7BF, analyzeCitrineBF, analyzeCitrineLTIBF, analyzeCitrineN7BF, analyzeTurquoiseBF, analyzeTurquoiseLTIBF, analyzeTurquoiseN7BF, analyzeBFPLTIBF, analyzeBFPN7BF]
fluorescent_functions = [analyzeRubyFLUOThallus, analyzeRubyFLUOSpore, analyzeHYPETFLUO, analyzeVenusFluoThallus, analyzeVenusFluoSpore, analyzeVenusLTIFluo, analyzeVenusN7Fluo, analyzeCitrineFluo, analyzeCitrineLTIFluo, analyzeCitrineN7Fluo, analyzeTurquoiseFluo, analyzeTurquoiseLTIFluo, analyzeTurquoiseN7Fluo, analyzeBFPLTIFluo, analyzeBFPN7Fluo]

all_areas = {}
all_intensities = {}

for fun in brightfield_functions:
    
    area_results = fun()
    all_areas[fun.__name__.rsplit('analyze', 1)[1]] = area_results # Split the function name, using the unique plant type to set as dictionary key
    
for fun in fluorescent_functions:
    
    intensity_results = fun()
    all_intensities[fun.__name__.rsplit('analyze', 1)[1]] = intensity_results # Split the function name, using the unique plant type to set as dictionary key

In [ ]:
import pandas as pd
from datetime import datetime

# Save the analyzed data to .csv file for export; save the file with date and time prefix to differentiate different analysis
curr_datetime = datetime.now()
curr_datetime_str = curr_datetime.strftime("%d-%m-%Y_%H-%M")

pd.DataFrame(all_areas).to_csv('/Users/Teoric/Desktop/{}_area'.format(curr_datetime_str), header=None, index=None)
pd.DataFrame(all_intensities).to_csv('/Users/Teoric/Desktop/{}_intensities'.format(curr_datetime_str), header=None, index=None)

In [ ]:
def plotSpecific(means=False, stddev=False):
    
    '''
    Plot the results for the area or instensity of a specific type of plant
    Inputs are an array which has to correspond to one of the names noted above and
    a string which has to be the name of the array converted to a string.
    
    Also can plot the mean and standard error for each day is means and stderr is set to True
    
    As an example:
    
    array = [...]
    name = 'array'
    '''
    #os.chdir('/Users/Teoric/Desktop/5DAYS/')
    
    print('Please enter one of the following names (exactly as written) to view the data: ')
    
    area_keys = all_areas.keys()
    intensity_keys = all_intensities.keys()
    
    for key in area_keys:
        
        print('{}'.format(key))
        
    for key in intensity_keys:
        
        print('{}'.format(key))
        
    key_used = str(input('Enter type of plant to analyze: '))
    
    if key_used in area_keys:
        
        array = all_areas['{}'.format(key_used)]
        
    elif key_used in intensity_keys:
        
        array = all_intensities['{}'.format(key_used)]
        
    else:
        
        print('Key not valid, please enter a valid key.')
        return None
    
    plt.figure(figsize=(7,7), dpi=250)
    plt.title(key_used+' over 5 days')
    x_axis = [[0 for i in range(len(array[0]))],[1 for i in range(len(array[1]))],[2 for i in range(len(array[2]))],[3 for i in range(len(array[3]))],[4 for i in range(len(array[4]))]]
    color_list = ['mediumvioletred', 'crimson', 'slateblue', 'rebeccapurple', 'indigo']
    
    
    if means==True and stddev==False:
        
        # Contains the info for the mean of area or intensity for each day
        means_per_day = []
        
        for day in range(0, 5):
        
            plt.scatter(x_axis[day], array[day], s=2., alpha=0.3, marker='o', c=color_list[day])

            #Calculate means

            mean_day = np.mean(array[day])
            means_per_day.append(mean_day)
            
            plt.scatter(day, mean_day, s=15., marker='D', c=color_list[day])

        if key_used in intensity_keys:

            plt.xlabel('Day')
            plt.ylabel('Intensity (arbitrary units)')

        elif key_used in area_keys:

            plt.xlabel('Day')
            plt.ylabel('Area (\u03BCm\u00B2)')

        return
        
    elif means == True and stddev==True:
        
        # Contains the info for the mean of area or intensity for each day
        means_per_day = []
        stddev_per_day = []
        
        for day in range(0, 5):
        
            plt.scatter(x_axis[day], array[day], s=2., alpha=0.3, marker='o', c=color_list[day])

            # Calculate means

            mean_day = np.mean(array[day])
            means_per_day.append(mean_day)
            
            plt.scatter(day, mean_day, s=15., marker='D', c=color_list[day])
            
            # Calculate Standard Deviation
            
            stddev = stats.sem(array[day])
            stddev_per_day.append(stddev)
            
            plt.errorbar(day, mean_day, yerr=stddev*2, ecolor=color_list[day], elinewidth=1., barsabove=True, capsize=3.)
            

        if key_used in intensity_keys:

            plt.xlabel('Day')
            plt.ylabel('Intensity (arbitrary units)')

        elif key_used in area_keys:

            plt.xlabel('Day')
            plt.ylabel('Area (\u03BCm\u00B2)')

        return
        
    else:
        
        for day in range(0, 5):
        
            plt.scatter(x_axis[day], array[day], s=4., marker='o')

        if key_used in intensity_keys:

            plt.xlabel('Day')
            plt.ylabel('Intensity (arbitrary units)')

        elif key_used in area_keys:

            plt.xlabel('Day')
            plt.ylabel('Area (\u03BCm\u00B2)')

        return

In [ ]:
# Plot the graph using the function defined in the previous cell

plotSpecific(means=True, stddev=False)

In [ ]:
from labellines import labelLine, labelLines

def plotSpecificLINES(array, name):
    
    '''
    Plot the results for the area or instensity of a specific type of plant
    Inputs are an array which has to correspond to one of the names noted above and
    a string which has to be the name of the array converted to a string.
    
    As an example:
    
    array = [...]
    name = 'array'
    '''
    os.chdir('/Users/alejandro/Desktop/')
    
    plt.figure(figsize=(7,7), dpi=250)
    plt.title(name+' over 5 days')
    x_axis = [0, 1, 2, 3, 4] # Number of days
    
    plantdata = [[] for i in range(0, len(array[0]))]
    plant_legend = []    
    
    for day in range(0, 5):
        
        for plantnumber in range(0, len(array[day])):
        
            plantdata[plantnumber].append(array[day][plantnumber])
            plant_legend.append(str(plantnumber+1))
            
    for i in range(0, len(plantdata)):
        
        plt.plot(list(range(len(plantdata[i]))), plantdata[i], marker='o', markersize=2.5)
        
    plt.legend(plant_legend, loc='upper left', ncol=2)

    if 'intensity' in name:

        plt.xlabel('Day')
        plt.ylabel('Intensity (arbitrary units)')

    elif 'area' in name:

        plt.xlabel('Day')
        plt.ylabel('Area (square micrometers)')
        
    return